In [ ]:
import numpy as np
import cupy as cp
from tqdm import tqdm
from PIL import Image
from collections import OrderedDict

#xt=0 #기억을 위한 전역변수
cp.cuda.Device(0).use()
#cp.cuda.Device(1).use()

# 유틸 im2col col2im 및 데이터 로더

In [ ]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = cp.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = cp.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    #print(col)
    return col

In [ ]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = cp.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

# SoftmaxWithLoss

In [ ]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - cp.max(x, axis=0)
        y = cp.exp(x) / cp.sum(cp.exp(x), axis=0)
        return y.T 

    x = x - cp.max(x) # 오버플로 대책
    return cp.exp(x) / cp.sum(cp.exp(x))

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
    
    batch_size = y.shape[0]
   
    return -cp.sum(cp.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)   
        self.loss = cross_entropy_error(self.y, self.t)
       
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[cp.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx

#레이어 테스트

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [ ]:
class Dropout:
    """
    http://arxiv.org/abs/1207.0580
    """
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = cp.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        return dout * self.mask

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = cp.dot(self.x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = cp.dot(dout, self.W.T)
        self.dW = cp.dot(self.x.T, dout)
        self.db = cp.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 중간 데이터（backward 시 사용）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 가중치와 편향 매개변수의 기울기
        self.dW = None
        self.db = None

    def forward(self, x):
        #print("컨볼루션 순전파")
        FN, C, FH, FW = self.W.shape
        #print("FN, C, FH, FW =>",FN, C, FH, FW)
        
        N, C, H, W = x.shape
        #print("N, C, H, W =>",N, C, H, W)
        
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = cp.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W
        #print("self.x =>",self.x)
        #print("self.col =>",self.col)
        #print("self.col_W.shape =>",self.col_W.shape)
        #print("self.col_W =>",self.col_W)
        #print(out.shape)
        return out

    def backward(self, dout):
        #print("컨볼루션 역전파")
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = cp.sum(dout, axis=0)
        self.dW = cp.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = cp.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

In [ ]:
class BatchNormalization:
    """
    http://arxiv.org/abs/1502.03167
    """
    def __init__(self, gamma, beta, momentum=0.9, running_mean=None, running_var=None):
        self.gamma = gamma
        self.beta = beta
        self.momentum = momentum
        self.input_shape = None # 합성곱 계층은 4차원, 완전연결 계층은 2차원  

        # 시험할 때 사용할 평균과 분산
        self.running_mean = running_mean
        self.running_var = running_var  
        
        # backward 시에 사용할 중간 데이터
        self.batch_size = None
        self.xc = None
        self.std = None
        self.dgamma = None
        self.dbeta = None

    def forward(self, x, train_flg=True):
        self.input_shape = x.shape
        if x.ndim != 2:
            N, C, H, W = x.shape
            x = x.reshape(N, -1)

        out = self.__forward(x, train_flg)
        
        return out.reshape(*self.input_shape)
            
    def __forward(self, x, train_flg):
        if self.running_mean is None:
            N, D = x.shape
            self.running_mean = cp.zeros(D)
            self.running_var = cp.zeros(D)
                        
        if train_flg:
            mu = x.mean(axis=0)
            xc = x - mu
            var = cp.mean(xc**2, axis=0)
            std = cp.sqrt(var + 10e-7)
            xn = xc / std
            
            self.batch_size = x.shape[0]
            self.xc = xc
            self.xn = xn
            self.std = std
            self.running_mean = self.momentum * self.running_mean + (1-self.momentum) * mu
            self.running_var = self.momentum * self.running_var + (1-self.momentum) * var            
        else:
            xc = x - self.running_mean
            xn = xc / ((cp.sqrt(self.running_var + 10e-7)))
            
        out = self.gamma * xn + self.beta 
        return out

    def backward(self, dout):
        if dout.ndim != 2:
            N, C, H, W = dout.shape
            dout = dout.reshape(N, -1)

        dx = self.__backward(dout)

        dx = dx.reshape(*self.input_shape)
        return dx

    def __backward(self, dout):
        dbeta = dout.sum(axis=0)
        dgamma = cp.sum(self.xn * dout, axis=0)
        dxn = self.gamma * dout
        dxc = dxn / self.std
        dstd = -cp.sum((dxn * self.xc) / (self.std * self.std), axis=0)
        dvar = 0.5 * dstd / self.std
        dxc += (2.0 / self.batch_size) * self.xc * dvar
        dmu = cp.sum(dxc, axis=0)
        dx = dxc - dmu / self.batch_size
        
        self.dgamma = dgamma
        self.dbeta = dbeta
        
        return dx


In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        #print("풀링 순전파")
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = cp.argmax(col, axis=1)
        out = cp.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max
        #print(out.shape)
        return out

    def backward(self, dout):
        #print("풀링 역전파")
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = cp.zeros((dout.size, pool_size))
        dmax[cp.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

#최적화 함수

In [ ]:
class SGD:

    """확률적 경사 하강법（Stochastic Gradient Descent）"""

    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key] 


class Momentum:

    """모멘텀 SGD"""

    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            params[key] += self.v[key]


class Nesterov:

    """Nesterov's Accelerated Gradient (http://arxiv.org/abs/1212.0901)"""
    # NAG는 모멘텀에서 한 단계 발전한 방법이다. (http://newsight.tistory.com/224)
    
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.v[key] *= self.momentum
            self.v[key] -= self.lr * grads[key]
            params[key] += self.momentum * self.momentum * self.v[key]
            params[key] -= (1 + self.momentum) * self.lr * grads[key]


class AdaGrad:

    """AdaGrad"""

    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)


class RMSprop:

    """RMSprop"""

    def __init__(self, lr=0.01, decay_rate = 0.99):
        self.lr = lr
        self.decay_rate = decay_rate
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] *= self.decay_rate
            self.h[key] += (1 - self.decay_rate) * grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)


class Adam:

    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            #self.m[key] = self.beta1*self.m[key] + (1-self.beta1)*grads[key]
            #self.v[key] = self.beta2*self.v[key] + (1-self.beta2)*(grads[key]**2)
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
            
            #unbias_m += (1 - self.beta1) * (grads[key] - self.m[key]) # correct bias
            #unbisa_b += (1 - self.beta2) * (grads[key]*grads[key] - self.v[key]) # correct bias
            #params[key] += self.lr * unbias_m / (np.sqrt(unbisa_b) + 1e-7)


#학습 함수 설정

In [ ]:
class Trainer:
    """신경망 훈련을 대신 해주는 클래스
    """
    def __init__(self, network, x_train, t_train, x_test, t_test,
                 epochs=150, mini_batch_size=100,
                 optimizer='SGD', optimizer_param={'lr':0.001}, 
                 evaluate_sample_num_per_epoch=None, verbose=True):
        self.network = network
        self.verbose = verbose
        self.x_train = x_train
        self.t_train = t_train
        self.x_test = x_test
        self.t_test = t_test
        self.epochs = epochs
        self.batch_size = mini_batch_size
        self.evaluate_sample_num_per_epoch = evaluate_sample_num_per_epoch

        # optimzer
        #optimizer_class_dict =  {'sgd':SGD} {'adam':Adam} adam 으로 하면 33번째에 W가 0이되어서 sgd로 변경함
        optimizer_class_dict = {'sgd':SGD}
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)
        
        self.train_size = x_train.shape[0]
        self.iter_per_epoch = max(self.train_size / mini_batch_size, 1)
        self.max_iter = int(epochs * self.iter_per_epoch)
        self.current_iter = 0
        self.current_epoch = 0
        
        self.train_loss_list = []
        self.train_acc_list = []
        self.test_acc_list = []


    def train_step(self):
        
        batch_mask = cp.random.choice(self.train_size, self.batch_size)
 

        x_batch = self.x_train[batch_mask]
        t_batch = self.t_train[batch_mask]
        
        grads = self.network.gradient(x_batch, t_batch)
        self.optimizer.update(self.network.params, grads)
        
        loss = self.network.loss(x_batch, t_batch)
        if self.verbose: print("train loss:" + str(loss))
        
        if self.current_iter % self.iter_per_epoch == 0:
            self.current_epoch += 1
            
            x_train_sample, t_train_sample = self.x_train, self.t_train
            x_test_sample, t_test_sample = self.x_test, self.t_test

            #if not (20 is None) -> if not false -> true 즉 evaluate_sample_num_per_epoch 이 지정되어 있으면  
            if not self.evaluate_sample_num_per_epoch is None: 
                t = self.evaluate_sample_num_per_epoch
                x_train_sample, t_train_sample = self.x_train[:t], self.t_train[:t]
                x_test_sample, t_test_sample = self.x_test[:t], self.t_test[:t]

            train_acc = self.network.accuracy(x_train_sample, t_train_sample)
            test_acc = self.network.accuracy(x_test_sample, t_test_sample)
            

            #loss = loss/10 #acc와 한개의 그래프로 나타내기 위해 10으로 나누어서 리스트에 저장함
            self.train_loss_list.append(loss) #loss값을 epoch 마다 한번씩만 기록
            print("self.train_loss_list:", self.train_loss_list)
            
            self.train_acc_list.append(train_acc)
            print("self.train_acc_list:", self.train_acc_list)
            
            self.test_acc_list.append(test_acc)
            print("self.test_acc_list:", self.test_acc_list)
            
            
            if self.verbose: 
                print("=== epoch:" + str(self.current_epoch) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc) + " ===")

        self.current_iter += 1

    def train(self):
        for i in tqdm(range(self.max_iter)):
            self.train_step()

        test_acc = self.network.accuracy(self.x_test, self.t_test)


        if self.verbose:
            print("=============== Final Test Accuracy ===============")
            print("test acc:" + str(test_acc))

#레이어 구성

In [ ]:
class test_net:
    def __init__(self, input_dim=(3, 224, 224),
                 conv_param_1 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':64, 'filter_size':3, 'pad':2, 'stride':1},
                
                 conv_param_3 = {'filter_num':128, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':128, 'filter_size':3, 'pad':1, 'stride':1},
                
                 conv_param_5 = {'filter_num':256, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':256, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_7 = {'filter_num':256, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_8 = {'filter_num':256, 'filter_size':3, 'pad':1, 'stride':1},
                 
                 conv_param_9 = {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_10 = {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_11 = {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_12 = {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':1},
                 
                 conv_param_13= {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_14= {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_15= {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_16= {'filter_num':512, 'filter_size':3, 'pad':1, 'stride':2},   
                 
                 hidden_size=4096, output_size=550):
        
        pre_node_nums = cp.array([3*3*3, 64*3*3, 64*3*3, 128*3*3, 128*3*3, 256*3*3, 256*3*3, 256*3*3,256*3*3, 512*3*3,512*3*3,512*3*3,512*3*3,512*3*3,512*3*3,512*3*3,hidden_size, hidden_size, hidden_size,output_size])
        wight_init_scales = cp.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
               
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6, conv_param_7, conv_param_8, conv_param_9, conv_param_10, conv_param_11, conv_param_12, conv_param_13, conv_param_14, conv_param_15, conv_param_16]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * cp.random.randn(conv_param['filter_num'], 
                                            pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = cp.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']

        
        self.params['W17'] = wight_init_scales[17] * cp.random.randn(512*7*7, hidden_size)
        self.params['b17'] = cp.zeros(hidden_size)
        
        self.params['W18'] = wight_init_scales[18] * cp.random.randn(hidden_size, hidden_size)
        self.params['b18'] = cp.zeros(hidden_size)
    
        self.params['W19'] = wight_init_scales[19] * cp.random.randn(hidden_size, output_size)
        self.params['b19'] = cp.zeros(output_size)
        
        
        ########################################################
        #self.layers = []
        # 계층 생성

        #레이어 쌓으면서 참고한 주소
        #https://kr.mathworks.com/help/deeplearning/ref/vgg19.html;jsessionid=140be85e056b065955d977e509ee

        self.layers = OrderedDict()

        #0 첫번째 layer 1_1
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], 
                                   conv_param_1['stride'], conv_param_1['pad'])
        #1 활성화 함수를 적용하기 전에 Normalization 
        self.layers['Batn1'] = BatchNormalization(1.0,0.0)
        self.layers['Relu1'] = Relu()
        
        #4 두번째 layer 1_2
        self.layers['Conv2'] = Convolution(self.params['W2'], self.params['b2'], 
                                   conv_param_2['stride'], conv_param_2['pad'])
        self.layers['Batn2'] = BatchNormalization(1.0,0.0)
        self.layers['Relu2'] = Relu()
        self.layers['Pool2'] = Pooling(pool_h=2, pool_w=2, stride=2)
     


        #8 세번째 layer 2_1
        self.layers['Conv3'] = Convolution(self.params['W3'], self.params['b3'], 
                                   conv_param_3['stride'], conv_param_3['pad'])    
        self.layers['Relu3'] = Relu()
        #self.layers.append(Relu())

        #10 네번째 layer 2_2
        self.layers['Conv4'] = Convolution(self.params['W4'], self.params['b4'],
                                   conv_param_4['stride'], conv_param_4['pad'])
        #11
        self.layers['Relu4'] = Relu()
        self.layers['Pool4'] = Pooling(pool_h=2, pool_w=2, stride=2)
     


        #12 다섯번째 layer 3_1
        self.layers['Conv5'] = Convolution(self.params['W5'], self.params['b5'],
                                   conv_param_5['stride'], conv_param_5['pad'])
        #13
        self.layers['Relu5'] = Relu()
        #self.layers.append(Relu())

        #15 여섯번째 layer 3_2
        self.layers['Conv6'] = Convolution(self.params['W6'], self.params['b6'],
                                   conv_param_6['stride'], conv_param_6['pad'])
        self.layers['Relu6'] = Relu()
        
        #16 일곱번째 layer 3_3
        self.layers['Conv7'] = Convolution(self.params['W7'], self.params['b7'],
                                   conv_param_7['stride'], conv_param_7['pad'])
        self.layers['Relu7'] = Relu()
             
        #17 여덜번째 layer 3_4
        self.layers['Conv8'] = Convolution(self.params['W8'], self.params['b8'],
                                   conv_param_8['stride'], conv_param_8['pad'])
        self.layers['Relu8'] = Relu()
        self.layers['Pool8'] = Pooling(pool_h=2, pool_w=2, stride=2)
           

          
        #18 아홉번째 layer 4_1
        self.layers['Conv9'] = Convolution(self.params['W9'], self.params['b9'],
                                   conv_param_9['stride'], conv_param_9['pad'])
        self.layers['Relu9'] = Relu()
        
        #19 열번째 layer 4_2
        self.layers['Conv10'] = Convolution(self.params['W10'], self.params['b10'],
                                   conv_param_10['stride'], conv_param_10['pad'])
        self.layers['Relu10'] = Relu()
        
        #20 열한번째 layer 4_3
        self.layers['Conv11'] = Convolution(self.params['W11'], self.params['b11'],
                                   conv_param_11['stride'], conv_param_11['pad'])
        self.layers['Relu11'] = Relu()
        
        #21 열두번째 layer 4_4
        self.layers['Conv12'] = Convolution(self.params['W12'], self.params['b12'],
                                   conv_param_12['stride'], conv_param_12['pad'])
        self.layers['Relu12'] = Relu()
        self.layers['Pool12'] = Pooling(pool_h=2, pool_w=2, stride=2)
         
  
         
        #22 열세번째 layer 5_1
        self.layers['Conv13'] = Convolution(self.params['W13'], self.params['b13'],
                                   conv_param_13['stride'], conv_param_13['pad'])
        self.layers['Relu13'] = Relu()
        
        #23 열네번째 layer 5_2
        self.layers['Conv14'] = Convolution(self.params['W14'], self.params['b14'],
                                   conv_param_14['stride'], conv_param_14['pad'])
        self.layers['Relu14'] = Relu()
        
        #24 열다섯번째 layer 5_3
        self.layers['Conv15'] = Convolution(self.params['W15'], self.params['b15'],
                                   conv_param_15['stride'], conv_param_15['pad'])
        self.layers['Relu15'] = Relu()
        
        #25 열여섯번째 layer  5_4
        self.layers['Conv16'] = Convolution(self.params['W16'], self.params['b16'],
                                   conv_param_16['stride'], conv_param_16['pad'])
        self.layers['Relu16'] = Relu()
        self.layers['Pool16'] = Pooling(pool_h=2, pool_w=2, stride=2)
      

      
        #26 열일곱번째 layer fc6
        self.layers['Affine17'] = Affine(self.params['W17'], self.params['b17'])
        self.layers['Relu17'] = Relu()
        self.layers['Drop17'] = Dropout(0.5)
                
        #27 열여덜번째 layer fc7
        self.layers['Affine18'] = Affine(self.params['W18'], self.params['b18'])
        self.layers['Relu18'] = Relu()      
        self.layers['Drop18'] = Dropout(0.5)
                
        #28 열아홉번째 layer fc8
        self.layers['Affine19'] = Affine(self.params['W19'], self.params['b19'])
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x, train_flg=False):
        #for layer in self.layers:
        for layer in self.layers.values():
            
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = cp.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = cp.argmax(y, axis=1)
            acc += cp.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
   
            
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Conv2'].dW, self.layers['Conv2'].db

        grads['W3'], grads['b3'] = self.layers['Conv3'].dW, self.layers['Conv3'].db
        grads['W4'], grads['b4'] = self.layers['Conv4'].dW, self.layers['Conv4'].db

        grads['W5'], grads['b5'] = self.layers['Conv5'].dW, self.layers['Conv5'].db 
        grads['W6'], grads['b6'] = self.layers['Conv6'].dW, self.layers['Conv6'].db
        grads['W7'], grads['b7'] = self.layers['Conv7'].dW, self.layers['Conv7'].db
        grads['W8'], grads['b8'] = self.layers['Conv8'].dW, self.layers['Conv8'].db

        grads['W9'], grads['b9'] = self.layers['Conv9'].dW, self.layers['Conv9'].db
        grads['W10'], grads['b10'] = self.layers['Conv10'].dW, self.layers['Conv10'].db
        grads['W11'], grads['b11'] = self.layers['Conv11'].dW, self.layers['Conv11'].db
        grads['W12'], grads['b12'] = self.layers['Conv12'].dW, self.layers['Conv12'].db

        grads['W13'], grads['b13'] = self.layers['Conv13'].dW, self.layers['Conv13'].db
        grads['W14'], grads['b14'] = self.layers['Conv14'].dW, self.layers['Conv14'].db
        grads['W15'], grads['b15'] = self.layers['Conv15'].dW, self.layers['Conv15'].db
        grads['W16'], grads['b16'] = self.layers['Conv16'].dW, self.layers['Conv16'].db


        grads['W17'], grads['b17'] = self.layers['Affine17'].dW, self.layers['Affine17'].db
        grads['W18'], grads['b18'] = self.layers['Affine18'].dW, self.layers['Affine18'].db
        grads['W19'], grads['b19'] = self.layers['Affine19'].dW, self.layers['Affine19'].db
                
        #if grads['W' + str(i+1)] == grads['W1']:
        #if 'W' + str(i+1) == 'W1':
        #print("grads['W1'] => ", grads['W1'])
        
        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
            
        for key, val in params.items():
            self.params[key] = val
        
        #for i, layer_idx in enumerate((0, 3, 7, 9, 12,15,18,21)):
        #for i, layer_idx in enumerate((0, 4, 8, 10, 12, 15, 18, 21)):
        for i, layer_idx in enumerate(['Conv1','Conv2','Conv3','Conv4','Conv5','Conv6','Conv7','Conv8','Conv9','Conv10','Conv11','Conv12','Conv13','Conv14','Conv15','Conv16','Affine17','Affine18','Affine19']):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]


        ##########################################################

#Input 시킬 부분  
## 일단 나중에 고려 

In [ ]:
#x_train = cp.load('face227_x_features_N_step9.pkl',mmap_mode = 'r+',allow_pickle = 'true')
#t_train = cp.load('face227_y_label_N_step9.pkl',mmap_mode = 'r+',allow_pickle = 'true')

#x_test = cp.load('face227_x_features_N_step10.pkl',mmap_mode = 'r+',allow_pickle = 'true')
#t_test = cp.load('face227_y_label_N_step10.pkl',mmap_mode = 'r+',allow_pickle = 'true')

#학습 시작

In [ ]:
network = test_net()
print(network.params['W1'].shape)
print(network.params['W2'].shape)
print(network.params['W3'].shape)
print(network.params['W4'].shape)
print(network.params['W5'].shape)
print(network.params['W6'].shape)
print(network.params['W7'].shape)
print(network.params['W8'].shape)
print(network.params['W9'].shape)
print(network.params['W10'].shape)
print(network.params['W11'].shape)
print(network.params['W12'].shape)
print(network.params['W13'].shape)
print(network.params['W14'].shape)
print(network.params['W15'].shape)
print(network.params['W16'].shape)
print(network.params['W17'].shape)
print(network.params['W18'].shape)
print(network.params['W19'].shape)
# 저장된 매개변수 사용
#network.load_params("params_simpleCNN_epoch300730.pkl")#original MNIST training, epoch 30
#network.load_params("alex_net_numpy_550_step"+t_1_str+"_GPU_5sgd_100.pkl") # step1~10
#network.load_params("alex_net_numpy_550_N_step"+t_1_str+"_GPU_sgd.pkl")
#alex_net_numpy_550_N_step1_GPU_sgd
print("loaded Network Parameters!")

#2곳수정 epochs=100, 
#3곳수정 mini_batch_size=100
#3곳 수정 optimizer='SGD''Adam'

trainer = Trainer(network, x_train, t_train, x_test, t_test, epochs=150, mini_batch_size=100, optimizer='SGD',optimizer_param={'lr':0.001} #optimizer_param={'lr':0.001}
                )

trainer.train()

#network.save_params("alex_net_numpy_550_step"+t_str+"_GPU_5sgd_100.pkl")
#network.save_params("alex_net_numpy_550_step"+t_str+"_GPU_5sgd_nouseparam_2.pkl")
#network.save_params("alex_net_numpy_symmetry_"+t_str+"_GPU_sgd.pkl")
#network.save_params("alex_net_numpy_550_N_step"+t_str+"_GPU_sgd.pkl")
#network.save_params("alex_net_numpy_227_step1_GPU_5.pkl")
#print("saved")

(64, 3, 3, 3)
(64, 64, 3, 3)
(128, 64, 3, 3)
(128, 128, 3, 3)
(256, 128, 3, 3)
(256, 256, 3, 3)
(256, 256, 3, 3)
(256, 256, 3, 3)
(512, 256, 3, 3)
(512, 512, 3, 3)
(512, 512, 3, 3)
(512, 512, 3, 3)
(512, 512, 3, 3)
(512, 512, 3, 3)
(512, 512, 3, 3)
(512, 512, 3, 3)
(25088, 4096)
(4096, 4096)
(4096, 550)


NameError: ignored